## MBA em Ciência de Dados
# Redes Neurais e Arquiteturas Profundas

## <span style="color:darkred">Avaliação Final</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

Nessa avaliação será utilizado o dataset `smartphone_activity.csv` que contém 561 colunas, cujo objetivo é classificar uma entre 6 ações de uma pessoa com base em sensores disponíveis no smartphone. Vamos assumir um cenário com alta disponibilidade de exemplos não rotulados, e baixa de exemplos rotulados. Para tal, pré-treinaremos camadas de uma rede neural com dados não anotados, a qual posteriormente será usada para compor um modelo inicial de classificação.

Conforme código abaixo, use como características de entrada as 561 primeiras colunas e como classe a última coluna (activity). 

As tarefas a realizar são as seguintes:

1. **Prepare** os dados conforme o código fornecido (leia abaixo e estude o código para entender):
    * carregue o dataset e organize as features e rótulos
    * conjunto S = 2% dos dados iniciais como treinamento com rótulo (assumiremos que temos rótulos apenas para esses 2%), no formato par (x,y)
    * conjunto U = 50% dos dados iniciais como treinamento não anotado (note que S está contido em U), 
    * conjunto T = o restante dos 50% para teste, no formato par (x,y).<br><br>
    
1.  (3,0 pt) **Modelo A**: projete e treine um undercomplete autoencoder com dropout na entrada para pré-treinamento baseado em auto-supervisão. Esse autoencoder aprende a preencher os valores eliminados na camada de entrada. A arquitetura deve ter a seguinte estrutura:
    1. Encoder:
        * entrada com 561 valores
        * dropout com taxa de 0.25
        * normalização em batch
        * densa 256 neurônios, relu
        * densa 256 neurônios, relu
        * densa 64 neurônios, ativação linear 
        * normalização em batch (consideraremos essa camada como sendo o código produzido pelo encoder)
        * ativação relu
        * dropout 0.25
    2. Decoder:
        * densa 256 neurônios, ativação tanh
        * densa 256 neurônios, ativação tanh
        * densa 561 neurônios, ativação tanh
    * Usar Adam com taxa de aprendizado inicial de 0.005 e com decaimento exponencial a -0.1<br>
    * Treinar com perda MSE por 20 épocas com batch size 16 utilizando o conjunto **U**<br>
    * Exiba a perda final MSE após as 20 utilizando o conjunto **U** (use o evaluate para isso)<br>
    * Obtenha o código (saída do encoder relativa a normalização em batch após a camada de 64 dimensões) para os dados de treinamento (conjunto **U**) e armazene-o num array `code_train`. Exiba na tela esse código para a primeira instância de treinamento (índice 0)<br><br>
    
1. (3,5 pt)  **Modelo B**: rede neural profunda densa, utilizando como base o encoder do modelo A (inclusive seus pesos pré-treinados), e inserindo uma nova camada densa de classificação (em 6 classes) com ativação softmax (essa inicializada aleatoriamente). Porém **não** deve conter o primeiro dropout (logo após a camada de entrada) do encoder, ou seja a primeira camada dropout deve ser removida.<br>
    * A arquitetura deve ter portanto as seguintes camadas:
        * entrada
        * normalização em batch
        * densa 256 neurônios, relu
        * densa 256 neurônios, relu
        * densa 64 neurônios, ativação linear
        * normalização em batch
        * ativação relu
        * dropout 0.25
        * densa 6 neurônios, softmax
        
    * Utilizar Adam com taxa de aprendizado inicial de 0.001 e com decaimento em todas as épocas exponencial a -0.1
    * Treinar com perda entropia cruzada categórica por 50 épocas com batch size 16 
    * Compute como métricas, além da perda, precisão e revocação (precision / recall)<br><br>
    
1. (3,5 pt) **Avalie a rede neural de classificação** (Modelo B): 
    * Exiba o gráfico da precisão e revocação calculada no treinamento ao longo das épocas para o modelo B
    * Exiba precisão e revocação calculada no treinamento S e teste T (use evaluate no modelo B)<br><br>
    
1. **Bônus:** (+1 ponto extra) 
    * (0,5) *Análise de projeção das características*: visualize scatterplots com os 2 principais componentes obtidos do PCA com as classes dos exemplos atribuídas com cores ou marcadores diferentes. Projetar em 2D os seguintes espaços:
        1. scatterplot com projeção PCA do conjunto de S original (561 dimensões)
        1. scatterplot com projeção PCA do código (64 dimensões) do conjunto S após processado pelo "encoder" do *Modelo A* (code_train obtido no item/questão 2)
        1. scatterplot com projeção PCA do código (64 dimensões) do conjunto S após processado pelo "encoder" do *Modelo B* (treinado no item/questão 3)<br><br>    
    
    * (0,5) Obtenha um classificador SVM com kernel linear, treinado nos dados S obtendo sua representação do código (64 dimensões) da rede de classificação (modelo B). Avalie precisão e revocação no treinamento S e teste T.

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy.random import seed
from tensorflow.random import set_seed
from tensorflow import keras
from tensorflow.keras import layers

2021-11-18 23:22:32.622327: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import pandas as pd
df = pd.read_csv("smartphone_activity_dataset.csv")
df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_553,feature_554,feature_555,feature_556,feature_557,feature_558,feature_559,feature_560,feature_561,activity
0,0.289,-0.0203,-0.1330,-0.995,-0.9830,-0.914,-0.995,-0.983,-0.924,-0.93500,...,-0.2990,-0.710,-0.1130,0.03040,-0.465,-0.0184,-0.841,0.180,-0.0586,5
1,0.278,-0.0164,-0.1240,-0.998,-0.9750,-0.960,-0.999,-0.975,-0.958,-0.94300,...,-0.5950,-0.861,0.0535,-0.00743,-0.733,0.7040,-0.845,0.180,-0.0543,5
2,0.280,-0.0195,-0.1130,-0.995,-0.9670,-0.979,-0.997,-0.964,-0.977,-0.93900,...,-0.3910,-0.760,-0.1190,0.17800,0.101,0.8090,-0.849,0.181,-0.0491,5
3,0.279,-0.0262,-0.1230,-0.996,-0.9830,-0.991,-0.997,-0.983,-0.989,-0.93900,...,-0.1170,-0.483,-0.0368,-0.01290,0.640,-0.4850,-0.849,0.182,-0.0477,5
4,0.277,-0.0166,-0.1150,-0.998,-0.9810,-0.990,-0.998,-0.980,-0.990,-0.94200,...,-0.3510,-0.699,0.1230,0.12300,0.694,-0.6160,-0.848,0.185,-0.0439,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10294,0.310,-0.0534,-0.0991,-0.288,-0.1410,-0.215,-0.356,-0.149,-0.232,0.18500,...,-0.3760,-0.751,-0.3370,0.34600,0.885,-0.6990,-0.652,0.275,0.1850,2
10295,0.363,-0.0392,-0.1060,-0.305,0.0281,-0.196,-0.374,-0.030,-0.270,0.18500,...,-0.3200,-0.700,-0.7370,-0.37300,-0.657,0.3230,-0.655,0.274,0.1820,2
10296,0.350,0.0301,-0.1160,-0.330,-0.0421,-0.250,-0.388,-0.133,-0.347,0.00747,...,-0.1190,-0.467,-0.1820,0.08860,0.697,0.3630,-0.655,0.274,0.1810,2
10297,0.238,0.0185,-0.0965,-0.323,-0.2300,-0.208,-0.392,-0.280,-0.289,0.00747,...,-0.2050,-0.618,0.4450,-0.81900,0.929,-0.0084,-0.660,0.265,0.1880,2


### Parte 1: separar dados

In [3]:
from tensorflow.keras.utils import to_categorical

rotulos = np.array(df['activity'])-1
features = np.array(df.iloc[:, :-1])

print(features.shape)
perc_train = 0.5
perc_rot = 0.02

n_train_U = int(features.shape[0]*perc_train)
n_train_S = int(features.shape[0]*perc_rot)
n_test = int(features.shape[0]*(1-perc_train))
print("Tamanho conjunto de treinamento: ", n_train_S)
print("Tamanho conjunto de treinamento não rotulado: ", n_train_U)
print("Tamanho conjunto de testes: ", n_test)

x_trainS = features[:n_train_S,:]
y_trainS = to_categorical(rotulos[:n_train_S], 6)
# rotulos discretos de treinamento
rot_trainS = rotulos[:n_train_S]

x_trainU = features[:n_train_U,:]
y_trainU = to_categorical(rotulos[:n_train_U], 6)

x_test = features[n_train_U:,:]
y_test = to_categorical(rotulos[n_train_U:], 6)
# rotulos discretos de teste
rot_test = rotulos[n_train_U:]

(10299, 561)
Tamanho conjunto de treinamento:  205
Tamanho conjunto de treinamento não rotulado:  5149
Tamanho conjunto de testes:  5149


### Parte 2: Modelo A

In [ ]:
seed(1)
set_seed(2)

### instanciar modelo AE
### compilar
### treinar

In [3]:
### avaliar MSE no treinamento

In [4]:
### obter código 64d da normalizacao em batch do treinamento
### exibir código da primeira instancia do treinamento

### Parte 3: Modelo B

In [6]:
### criar novo modelo usando encoder pré-treinado do modelo A

# dica: obter lista de camadas
#layers = [l for l in modelo.layers]

# montar novo modelo com nova camada de entrada
# adicionar camadas anteriores usando novo_modelo.add(layers[i])
# adicionar novas camadas

In [7]:
# compilar modelo B
# treinar modelo B

### Parte 4: Avaliação da rede neural de classificação

In [8]:
# exibir grafico com precision e recall das épocas

In [9]:
# avalie precision e recall final no treinamento/teste

### Bônus

#### 1: Análise do espaço de características aprendido (0,5 pt)

In [10]:
from sklearn.decomposition import PCA

In [ ]:
# código exemplo para scatterplot (sendo pca_train o array com dados projetados, e rot_train rotulos discretos)
scatter = ax.scatter(pca_train[:,0], pca_train[:,1], c=rot_train, cmap="jet")
legend1 = ax.legend(*scatter.legend_elements(), loc="upper right", title="Classes")
ax.add_artist(legend1)
plt.title('PCA')

#### 2: SVM no código aprendido (0,5 pt)

In [12]:
from sklearn import svm